NYC Parking Data Cleaning
==========
(Take 2)
------

# 1. Obtain the data

In [5]:
# NYC LION street Lines
! cd data && curl -O "https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyclion_18a.zip" && unzip nyclion_18a.zip && rm nyclion_18a.zip

# NYC Street Parking Signs
! cd data && curl -O "http://a841-dotweb01.nyc.gov/datafeeds/ParkingReg/Parking_Regulation_Shapefile.zip" && unzip Parking_Regulation_Shapefile.zip && rm Parking_Regulation_Shapefile.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.8M  100 21.8M    0     0  1629k      0  0:00:13  0:00:13 --:--:-- 1926k
Archive:  Parking_Regulation_Shapefile.zip
  inflating: Parking_Regulation_Shapefile/Parking Regulation Metadata.pdf  
 extracting: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.CPG  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.dbf  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.prj  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.sbn  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.sbx  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp.xml  
  inflating: Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shx  


In [127]:
! pip install -U fiona dateutil
! pip install pyqtree
! brew upgrade gdal
! brew install spatialindex

Requirement already up-to-date: fiona in /anaconda3/lib/python3.6/site-packages
Requirement already up-to-date: six in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: click-plugins in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: cligj in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: munch in /anaconda3/lib/python3.6/site-packages (from fiona)
Requirement already up-to-date: click>=3.0 in /anaconda3/lib/python3.6/site-packages (from click-plugins->fiona)
You are using pip version 9.0.2, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core, caskroom/cask).
No changes to formulae.

Error: gdal 2.2.4 already installed
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
You 

In [1]:
import pandas as pd
import geopandas as gpd
import re
from dateutil.rrule import rrule, WEEKLY
from datetime import datetime

In [27]:
signs = gpd.read_file('data/Parking_Regulation_Shapefile/Parking_Regulation_Shapefile.shp')

In [3]:
aspsigns = signs[signs.SIGNDESC1.str.contains('\(SANITATION BROOM SYMBOL\)')]

In [4]:
aspsigns.SIGNDESC1.value_counts()

NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM MON & THURS <---->                                                    3955
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM TUES & FRI <---->                                                     3928
NO PARKING (SANITATION BROOM SYMBOL) TUESDAY FRIDAY 11:30AM-1PM <->                                                       3638
NO PARKING (SANITATION BROOM SYMBOL) MONDAY THURSDAY 11:30AM-1PM <->                                                      3567
NO PARKING (SANITATION BROOM SYMBOL) 9:30-11AM MON & THURS <--->                                                          3011
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM TUES <---->                                                           2978
NO PARKING (SANITATION BROOM SYMBOL) 11AM TO 12:30PM TUES & FRI <---->                                                    2914
NO PARKING (SANITATION BROOM SYMBOL) 11:30AM TO 1PM MON <--->                                                  

In [12]:
aspsigns[aspsigns.SIGNDESC1.str.contains("EXCEPT")].SIGNDESC1.value_counts()

NO PARKING (SANITATION BROOM SYMBOL) 7:30AM-8AM EXCEPT SUNDAY <->                                                     2615
NO PARKING (SANITATION BROOM SYMBOL) 8AM-8:30AM EXCEPT SUNDAY <->                                                     2426
NO PARKING (SANITATION BROOM SYMBOL) 8:30AM-9AM EXCEPT SUNDAY <->                                                     1502
NO PARKING (SANITATION BROOM SYMBOL) 7:30AM-8AM EXCEPT SUNDAY -->                                                      988
NO PARKING (SANITATION BROOM SYMBOL) 8AM-8:30AM EXCEPT SUNDAY -->                                                      929
NO PARKING (SANITATION BROOM SYMBOL) 8:30AM-9AM EXCEPT SUNDAY -->                                                      610
NO PARKING (SANITATION BROOM SYMBOL) 7AM-7:30AM EXCEPT SUNDAY <->                                                      299
NO PARKING (SANITATION BROOM SYMBOL) 8AM-9AM EXCEPT SUNDAY <->                                                         287
NO PARKING (SANI

## What is the data model for a parking restriction?

* Street Segment
* Day(s) of week
* Time of Restriction (on those days)


In [120]:
TIMERE = re.compile('(\d+:?\d*[A|P]?M?|NOON|MIDNIGHT)(?:-| TO | | TO-|)(\d+:?\d*[A|P]?M?|NOON|MIDNIGHT)')
MIDDAY = re.compile('([A|P]M)')
EXCEPTDAYS = re.compile('EXCEPT (MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)+')
RANGEDAYS = re.compile('(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)-(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)')
ALLDAYS = re.compile('(MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY|MON|TUES|WED|THURS|FRI|SAT|SUN)')

In [7]:
TIMERE.findall('NO PARKING (SANITATION BROOM SYMBOL) 10:30AM TO NOON MON & THURS W/SINGLE ARROW')

[('10:30AM', 'NOON')]

In [21]:
smp = aspsigns.sample(1000)

In [22]:


for ix, row in smp.iterrows():

    try:
        timeFrom, timeTo = TIMERE.findall(row['SIGNDESC1'])[0]
    except:
        print(row)
        raise
    if timeFrom == "NOON": timeFrom = "12:00PM"
    if timeFrom == "MIDNIGHT": timeFrom = "12:00AM"
    if timeTo == "NOON": timeTo = "12:00PM"
    if timeTo == "MIDNIGHT": timeTo = "12:00AM"
    if not MIDDAY.search(timeFrom) and MIDDAY.search(timeTo):
        timeFrom += (MIDDAY.findall(timeTo)[0])
        

In [8]:
def timeRules(signdesc):
    try:
        timeFrom, timeTo = TIMERE.findall(signdesc)[0]
    except:
        print(signdesc)
        raise
    if timeFrom == "NOON":
        timeFrom = "12:00PM"
    if timeFrom == "MIDNIGHT":
        timeFrom = "12:00AM"
    if timeTo == "NOON":
        timeTo = "12:00PM"
    if timeTo == "MIDNIGHT":
        timeTo = "12:00AM"
    if not MIDDAY.search(timeFrom) and MIDDAY.search(timeTo):
        timeFrom += (MIDDAY.findall(timeTo)[0])
    
    return timeFrom, timeTo

In [112]:
NORMALIZEDAYS = {
    'SUNDAY': 'SUNDAY',
    'SUN': 'SUNDAY',
    'MONDAY': 'MONDAY',
    'MON': 'MONDAY',
    'TUESDAY': 'TUESDAY',
    'TUE': 'TUESDAY',
    'TUES': 'TUESDAY',
    'WEDNESDAY': 'WEDNESDAY',
    'WED': 'WEDNESDAY',
    'WEDS': 'WEDNESDAY',
    'THURSDAY': 'THURSDAY',
    'THU': 'THURSDAY',
    'THUR': 'THURSDAY',
    'THURS': 'THURSDAY',
    'FRIDAY': 'FRIDAY',
    'FRI': 'FRIDAY',
    'SATURDAY': 'SATURDAY',
    'SAT': 'SATURDAY'
}
def dayRules(signdesc):
    exceptday = EXCEPTDAYS.findall(signdesc)
    rangeday = RANGEDAYS.findall(signdesc)
    alldays = ALLDAYS.findall(signdesc)

    dayslist = ['SUNDAY', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
    if exceptday:
        dayslist = ['SUNDAY', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
        
        for d in exceptday:
            day = NORMALIZEDAYS[d]
            dayslist.remove(day)
        return dayslist
    elif rangeday:
        #print(rangeday)
        fromDay, toDay = rangeday[0]
        fromIx = dayslist.index(fromDay)
        toIx = dayslist.index(toDay)
        #print(fromIx,toIx)
        return dayslist[fromIx:toIx+1]
    else:
        return alldays
        
        
    
        

In [143]:
for i in vc.index:
    days = dayRules(i)
    try:
        fromTime, toTime = timeRules(i)
    except:
        print(i)
        pass


NO PARKING (SANITATION BROOM SYMBOL) <->
NO PARKING (SANITATION BROOM SYMBOL) <->


# Side Quest: Lets do topic modeling on these sign texts!

Lets look at the unique text of each sign 

In [158]:
count_signs = signs.groupby(['SG_MUTCD_C', 'SIGNDESC1']).size().reset_index(name="Count").sort_values(by="Count", ascending=False)

In [170]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
documents = list(count_signs['SIGNDESC1'])

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

no_topics = 50

# Run NMF
# nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 5
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
available description www logo visit
Topic 1:
taxi stand limit symbol hour
Topic 2:
driveway clear broom symbol sanitation
Topic 3:
tunnel bridge tbta august crosswalk
Topic 4:
large lottery stand panel permit
Topic 5:
star symbol avo dept nyc
Topic 6:
arrow sign symbol clock right
Topic 7:
truck symbol loading 6pm friday
Topic 8:
curve president chevron studios hazard
Topic 9:
muni meter pay vehicle marshals
Topic 10:
7pm parking hour 9am metered
Topic 11:
hour 7pm parking 30am metered
Topic 12:
market farmers 30 nov sun
Topic 13:
parking symbol sanitation broom 30am
Topic 14:
information dot parking emergency telecommunications
Topic 15:
tour park layover slippery saturday
Topic 16:
7pm standing monday friday 10am
Topic 17:
diamond red delineator pictorial inch
Topic 18:
mon fri parking standing 8am
Topic 19:
broom 30am symbol parking sanitation
Topic 20:
11am 30pm 12 symbol crossing
Topic 21:
office general mayor justice medical
Topic 22:
city day times bus xyy
Topic 23:
co

# Start matching sign positions to street segments

Steps:
1. Determine the logic for matching a side to each street
1. For each Sign: match sign to closest street segment (and save it to a list on the correct sign)
2. For each street segment: apply the set of street signs (we have logic for) to determine the rules that apply to that street segment

In [198]:
# Convert LION Geodatabase to a GeoJSON file because Geopandas has FEELINGS! about importing geodatabase layers
! ogr2ogr -f "GeoJSON" data/lion/lion.geojson data/lion/lion.gdb lion -s_srs EPSG:2263 -t_srs EPSG:4326

In [2]:
streets = gpd.read_file('data/lion/lion.geojson')

Start your script by importing the quad tree.

from pyqtree import Index
Setup the spatial index, giving it a bounding box area to keep track of. The bounding box being in a four-tuple: (xmin, ymin, xmax, ymax).

spindex = Index(bbox=(0, 0, 100, 100))
Populate the index with items that you want to be retrieved at a later point, along with each item's geographic bbox.

# this example assumes you have a list of items with bbox attribute
for item in items:
    spindex.insert(item, item.bbox)
Then when you have a region of interest and you wish to retrieve items from that region, just use the index's intersect method. This quickly gives you a list of the stored items whose bboxes intersects your region of interests.

overlapbbox = (51, 51, 86, 86)
matches = spindex.intersect(overlapbbox)

In [17]:
! pip install -U rtree

Requirement already up-to-date: rtree in /anaconda3/lib/python3.6/site-packages
Requirement already up-to-date: setuptools in /anaconda3/lib/python3.6/site-packages (from rtree)
You are using pip version 9.0.2, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import rtree

In [24]:

spindex = rtree.index.Index()

In [25]:
%%time
a = 0
for ix, row in streets.iterrows():
    spindex.insert(ix, row[streets._geometry_column_name].bounds)
    a +=1
print(a)

224762
CPU times: user 47.9 s, sys: 792 ms, total: 48.7 s
Wall time: 48.7 s


## Now that you've built a spatial index, query against each point


In [56]:
for ix, row in signs.iterrows():
    break

In [57]:
ptx = srow[signs._geometry_column_name].centroid.coords.xy[0][0]
pty = srow[signs._geometry_column_name].centroid.coords.xy[1][0]

In [60]:
l = list(spindex.nearest((ptx, pty),num_results=10))
tencloseststreets = streets.iloc[l]

In [69]:
%%time
sample = signs.sample(1000)


CPU times: user 13.4 ms, sys: 1.44 ms, total: 14.9 ms
Wall time: 13.2 ms


In [77]:
def min_distance(point, lines):
    l = list(spindex.nearest(point,num_results=10))
    tencloseststreets = streets.iloc[l]
    return tencloseststreets #.distance(point).min()

# sample['min_dist_to_lines'] = sample.geometry.apply(lambda x: min_distance(x, streets))

In [85]:
tc = min_distance((ptx,pty), streets)
tc['min_dist'] = tc.distance(srow[signs._geometry_column_name].centroid).min()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [86]:
tc

,Street,SAFStreetName,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,SpecAddr,...,RLo_Hyphen,RHi_Hyphen,FromLeft,ToLeft,FromRight,ToRight,Join_ID,SHAPE_Length,geometry,min_dist
11812,JEROME AVENUE,,0,U,,B,,T,DOT,,...,2614,2666,2641,2669,2614,2666,2410001000000,315.085388,(LINESTRING (-73.89797842694803 40.86672770727...,0.000131
35614,JEROME AVENUE LINE,,1,U,,B,,,,,...,,,0,0,0,0,2504806010000,315.085388,(LINESTRING (-73.89797842694803 40.86672770727...,0.000131
17127,WEST KINGSBRIDGE ROAD,,0,U,,B,,T,DOT,,...,1,27,2,24,1,27,2419501000000,323.901438,(LINESTRING (-73.89740323381223 40.86747418930...,0.000131
6637,EAST KINGSBRIDGE ROAD,,0,U,,B,,T,DOT,,...,2,36,1,43,2,36,2419001000000,340.721052,(LINESTRING (-73.89740323381223 40.86747418930...,0.000131
1249,JEROME AVENUE,,0,U,,B,,T,DOT,,...,2670,2708,2671,2709,2670,2708,2410001000000,460.781340,(LINESTRING (-73.89740323381223 40.86747418930...,0.000131
36045,JEROME AVENUE LINE,,1,U,,B,,,,,...,,,0,0,0,0,2504806010000,460.781340,(LINESTRING (-73.89740323381223 40.86747418930...,0.000131
17012,EAST 193 STREET,,0,U,,B,,T,DOT,,...,2,38,1,39,2,38,2262801000000,339.001784,(LINESTRING (-73.89797842694803 40.86672770727...,0.000131
35624,JEROME AVENUE LINE,,1,U,,B,,,,,...,,,0,0,0,0,2504806010000,484.589229,(LINESTRING (-73.89887282939888 40.86558402623...,0.000131
6674,JEROME AVENUE,ST JAMES PARK,0,U,,B,,T,DOT,N,...,,,0,0,0,0,21209001000000N,484.589229,(LINESTRING (-73.89887282939888 40.86558402623...,0.000131
6675,JEROME AVENUE,ST JAMES PARK PLAYGROUND,0,U,,B,,T,DOT,G,...,,,0,0,0,0,21209004000000G,484.589229,(LINESTRING (-73.89887282939888 40.86558402623...,0.000131
